# The code for the "song recommender" app

In [1]:
# import relevant libraries
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re
import random
import config

In [2]:
top_100 = pd.read_csv("top_100.csv", index_col=False)

In [3]:
top_100.head()

,Unnamed: 0,Artist,Song
0,0,Miley Cyrus,Flowers
1,1,SZA,Kill Bill
2,2,Morgan Wallen,Last Night
3,3,Sam Smith & Kim Petras,Unholy
4,4,"Metro Boomin, The Weeknd & 21 Savage",Creepin'


In [4]:
# writing the song recommender function
def song_recommender():
    """Function that prompts the user for song input
    and checks whether it's in the top-100 csv file."""
    top_100 = pd.read_csv("top_100.csv")
    user_input = input("Please type in your favorite song: ")
    if user_input.lower() in map(str.lower, list(top_100["Song"])):
        print("Your song is in the current top-100.")
        random_song = random.choice(list(top_100["Song"]))
        artist = top_100.loc[top_100["Song"] == random_song, 'Artist'].to_string(index=False)
        print(f"\nHere is another song from the top-100 you might like: {random_song} by {artist}")
    else: 
        print("Your song is not in the current top-100. Let me give you a suggestion for another song.")

In [5]:
# writing the while loop that ensures the programme keeps running
while True:
    song_recommender()
    user_input = input("\nDo you want to input another song? yes/no: ")
    if user_input == "yes":
        continue
    elif user_input == "no":
        print("Your session has ended. Thank you for using our app!")
        break
    else:
        print("Please write either 'yes' or 'no'")
        continue

Please type in your favorite song: Flowers
Your song is in the current top-100.

Here is another song from the top-100 you might like: Never Gonna Not Dance Again by P!nk

Do you want to input another song? yes/no: no
Your session has ended. Thank you for using our app!


# Creating the song database with audio features

In [6]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


# initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [7]:
# checking a playlist
playlist = sp.user_playlist_tracks("spotify", "27gN69ebwiJRtXEboL12Ih")
playlist["items"][0]

{'added_at': '2022-11-26T02:31:48Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/celsum76'},
  'href': 'https://api.spotify.com/v1/users/celsum76',
  'id': 'celsum76',
  'type': 'user',
  'uri': 'spotify:user:celsum76'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5M52tdBnJaKSvOpJGz8mfZ'},
     'href': 'https://api.spotify.com/v1/artists/5M52tdBnJaKSvOpJGz8mfZ',
     'id': '5M52tdBnJaKSvOpJGz8mfZ',
     'name': 'Black Sabbath',
     'type': 'artist',
     'uri': 'spotify:artist:5M52tdBnJaKSvOpJGz8mfZ'}],
   'available_markets': [],
   'external_urls': {'spotify': 'https://open.spotify.com/album/1JA2UhLRbFRkmoh6Lz64KH'},
   'href': 'https://api.spotify.com/v1/albums/1JA2UhLRbFRkmoh6Lz64KH',
   'id': '1JA2UhLRbFRkmoh6Lz64KH',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b2738c9131b3b0f5a4b9fdc5a0ac',
    

In [8]:
# function that retrieves the songs from a playlist
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [9]:
# creating a list with playlists id's and defining the variable 'tracks' to store the results
playlist_ids = ["27gN69ebwiJRtXEboL12Ih", "37i9dQZF1DX4pUKG1kS0Ac", "37i9dQZF1DXbITWG1ZJKYt", "37i9dQZF1DXbSbnqxMTGx9", "37i9dQZF1DX8a1tdzq5tbM", "37i9dQZF1EQpj7X7UK8OOF", "37i9dQZF1EQqkOPvHGajmW", "17IFbN8moTMWsaK5S5qCyD"]
tracks = []
# genres included: metal, guilty pleasures, jazz, reggae, dance, rock, indie, rap/hiphop

# for loop that loops through the playlist ids and retrieves the song with the get_playlist_tracks function
for x in playlist_ids:
    tracks += get_playlist_tracks("spotify", x)

In [10]:
# checking how many tracks are in the Pandas dataframe
print(len(tracks))

1043


In [11]:
# getting the audio features
list_of_audio_features = []
for item in range(len(tracks)):
    #print (tracks[item]["track"]["id"])
    list_of_audio_features.append(sp.audio_features(tracks[item]["track"]["id"])[0])

In [12]:
print(len(tracks))

1043


In [13]:
song_df = pd.DataFrame(list_of_audio_features)    
song_df = song_df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

song_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.423,0.685,-9.651,0.0780,0.000063,0.006420,0.1330,0.328,162.780,3q9G9SvgdfE1fxlvoDIb20,167720
1,0.324,0.991,-4.465,0.1970,0.002470,0.008950,0.1040,0.195,140.245,47FyQCd3TYLrZ9TU6MPaWK,166707
2,0.308,0.866,-5.671,0.0590,0.000739,0.001410,0.2320,0.227,109.825,6p8eFfPw3nQkf37aT3AkmK,438120
3,0.453,0.905,-4.001,0.1150,0.360000,0.000569,0.2600,0.474,137.571,2b9lp5A6CqSzwOrBfAFhof,296200
4,0.420,0.930,-5.583,0.0638,0.018900,0.006750,0.2640,0.344,99.641,1s4Ie0cT6P73SRSfh3oyGW,291373
...,...,...,...,...,...,...,...,...,...,...,...
1038,0.794,0.614,-5.352,0.0467,0.096400,0.000000,0.0788,0.663,80.569,7jLbTp3qZzah9kMIdW8e5M,238053
1039,0.780,0.575,-7.247,0.2730,0.430000,0.000000,0.1430,0.773,84.492,2g8HN35AnVGIk7B8yMucww,252747
1040,0.748,0.614,-6.004,0.4070,0.067200,0.000078,0.1060,0.570,172.030,36UjTTrSwaAqLkAlf2Ooc4,230493
1041,0.731,0.807,-6.492,0.1000,0.181000,0.751000,0.3190,0.765,114.759,2bLqfJjuC5syrsgDsZfGmn,179120


In [14]:
song_df.to_csv("songs_df.csv", index=False)

In [15]:
# goal of today is to get a lot of songs into a Pandas dataframe with the associated audio features.
# sp search, getting id of song, getting audio features. Putting it in the right order.
# first try out the code with a small amount songs. Couple of thousands is already okay, but the more the better.